## CPSC 477 Final Project Part 2: Finetuning Mistral 7B-Instruct

Followed this tutorial: https://adithyask.medium.com/a-beginners-guide-to-fine-tuning-mistral-7b-instruct-model-0f39647b20fe

Imported `dataset/train.csv`

In [1]:
!pip install -q torch
!pip install -q git+https://github.com/huggingface/transformers #huggingface transformers for downloading models weights
!pip install -q datasets #huggingface datasets to download and manipulate datasets
!pip install -q peft #Parameter efficient finetuning - for qLora Finetuning
!pip install -q bitsandbytes #For Model weights quantisation
!pip install -q trl #Transformer Reinforcement Learning - For Finetuning using Supervised Fine-tuning
!pip install -q wandb -U #Used to monitor the model score during training

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer

In [3]:
notebook_login()

In [4]:
train_dataset = load_dataset('csv', data_files='./train.csv' , split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
print(len(train_dataset[0]["text"]))

9487


In [9]:
test_dataset = load_dataset('csv', data_files='./test.csv' , split='train')

In [10]:
print(len(test_dataset))

50


In [6]:
new_model = "mistralai-Code-Instruct" #set the name of the new model

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 1

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [7]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load the base model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)

base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

# Load MitsralAi tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

296

In [ ]:
eval_prompt = train_dataset[0]["text"].split("#")[0] + "# "

# import random
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

base_model.eval()
with torch.no_grad():
    print(tokenizer.decode(base_model.generate(**model_input, max_new_tokens=256, pad_token_id=2)[0], skip_special_tokens=True))

[INST] You are a financial advisor tasked with creating a short summary of an earnings call transcript. You only want to summarize or re-iterate points that would be relevant, critical, or informational to someone who wants to skim over the important details of a long transcript. Below is an earnings call transcript.

Earnings Call Transcript:
 On the call are Jeff Mezger, Chairman, President and Chief Executive Officer; Matt Mandino, Executive Vice President and Chief Operating Officer; Jeff Kaminski, Executive Vice President and Chief Financial Officer; Bill Hollinger, Senior Vice President and Chief Accounting Officer; and Thad Johnson, Senior Vice President and Treasurer.
And with that, here is Jeff Mezger.
We delivered healthy results in the second quarter marked by one of the strongest quarters for both operating and gross margin performance in some time.
Operationally, our divisions are doing an excellent job of navigating this environment of demand strength and well-publicized 

In [ ]:
print(train_dataset[0]["text"].split("#")[1])
print(len(train_dataset[0]["text"]))

 KB Home delivered a strong second quarter, driven by solid operational performance and favorable market conditions. The company achieved diluted earnings per share of $1.50, exceeding analysts' estimates. Total revenues rose 58% to $1.44 billion, largely attributed to a surge in housing revenues, which witnessed a 40% increase. KB Home experienced a robust surge in net orders, reaching a 14-year high for the second quarter, and its backlog grew significantly by 126% to $4.3 billion. The company's profitability metrics improved, with an operating income margin of 11.3% and a gross profit margin of 21.4%. This success was driven by disciplined pricing, operating leverage, and lower interest amortization. Despite facing challenges with supply shortages and municipal delays, KB Home remains confident in achieving its full-year targets. The company anticipates revenues ranging from $5.9 billion to $6.1 billion and expects to expand its operating margin to a range of 11.5% to 12%. KB Home r

In [12]:
with open("base_model_inference/test.txt", "w") as f:
    f.write("test")

In [16]:
# Run base model inference on test dataset
from tqdm import tqdm

base_model.eval()
with torch.no_grad():
    for i in range(len(test_dataset)):
        print(test_dataset[i]["filename"])
        eval_prompt = test_dataset[i]["text"].split("#")[0] + "# "
        model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
        prompt_and_summary = tokenizer.decode(base_model.generate(**model_input, max_new_tokens=500, pad_token_id=2)[0], skip_special_tokens=True)
        summary = prompt_and_summary.split("#")[1].strip()
        with open(f"base_model_inference/{test_dataset[i]['filename']}", "w") as f:
            f.write(summary)

WSO_q2_2021.txt
OFC_q2_2021.txt
LIN_q3_2021.txt
CNC_q3_2021.txt
SKT_q1_2021.txt
CHE_q4_2020.txt
MBI_q4_2019.txt
GHL_q4_2020.txt
KFY_q2_2022.txt
WST_q1_2020.txt
UVE_q4_2020.txt
NNN_q1_2021.txt
CHE_q3_2020.txt
GEF_q3_2021.txt
AIT_q2_2020.txt
TXT_q1_2021.txt
ATR_q1_2021.txt
LAZ_q2_2021.txt
HZO_q1_2021.txt
PBH_q3_2021.txt
HII_q1_2021.txt
PNW_q4_2019.txt
GD_q1_2021.txt
HCC_q1_2020.txt
CIM_q1_2020.txt
SJI_q3_2021.txt
FR_q4_2020.txt
TXT_q4_2021.txt
WST_q3_2020.txt
PLD_q4_2021.txt
FN_q2_2021.txt
NHI_q1_2020.txt
LHX_q1_2021.txt
HPP_q3_2021.txt
LH_q3_2021.txt
NX_q1_2021.txt
HVT_q1_2021.txt
AIR_q1_2022.txt
JBL_q1_2022.txt
MSI_q3_2021.txt
OXM_q2_2021.txt
TDY_q4_2020.txt
SPG_q1_2021.txt
GHM_q2_2021.txt
WEC_q4_2021.txt
PKG_q4_2020.txt
KWR_q1_2020.txt
NI_q3_2021.txt
KMB_q3_2020.txt
FNF_q3_2020.txt


In [17]:
!zip -r base_model_inference.zip base_model_inference/

  adding: base_model_inference/ (stored 0%)
  adding: base_model_inference/HCC_q1_2020.txt (deflated 58%)
  adding: base_model_inference/CHE_q3_2020.txt (deflated 49%)
  adding: base_model_inference/HVT_q1_2021.txt (deflated 44%)
  adding: base_model_inference/HII_q1_2021.txt (deflated 52%)
  adding: base_model_inference/LAZ_q2_2021.txt (deflated 55%)
  adding: base_model_inference/GHL_q4_2020.txt (deflated 54%)
  adding: base_model_inference/LH_q3_2021.txt (deflated 50%)
  adding: base_model_inference/AIT_q2_2020.txt (deflated 46%)
  adding: base_model_inference/WEC_q4_2021.txt (deflated 51%)
  adding: base_model_inference/CNC_q3_2021.txt (deflated 35%)
  adding: base_model_inference/PLD_q4_2021.txt (deflated 49%)
  adding: base_model_inference/TXT_q1_2021.txt (deflated 54%)
  adding: base_model_inference/NNN_q1_2021.txt (deflated 57%)
  adding: base_model_inference/KFY_q2_2022.txt (deflated 46%)
  adding: base_model_inference/PNW_q4_2019.txt (deflated 49%)
  adding: base_model_infere

In [18]:
from google.colab import files
files.download('base_model_inference.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Set LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=100, # the total number of training steps to perform
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Initialize the SFTTrainer for fine-tuning
trainer = SFTTrainer(
    model=base_model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,  # You can specify the maximum sequence length here
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
import gc
import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()
gc.collect()

# Start the training process
trainer.train()

# Save the fine-tuned model
trainer.model.save_pretrained(new_model)

Step,Training Loss
25,2.015200
50,1.925400
75,1.851100
100,1.846500


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers fou